## Initialise RAG

In [2]:

from RAG.librarian import Librarian 

librarian = Librarian(librarian_LLM_model = "GEMINI")

# SELECT SPECIALIST DATABASE
librarian.select_specialist(specialist = "traveller", specialist_LLM_model = "GEMINI", )

# Ask librarian to get acquinted with the specialist database
librarian.Traveller.load_data_model(reembed = False,
                                    embed_id = 0,
                                    data_model_keys = {"TEST - CLIENT":"CLIENT ID",
                                                        "TEST - CLIENT REQUEST":"CLIENT ID",
                                                        "TEST - FLIGHTS":"FLIGHT ID",
                                                        "TEST - ACCOMODATIONS":"ACCOMODATION ID",
                                                        "TEST - ACTIVITIES":"ACTIVITY ID",
                                                        "TEST - SERVICES":"SERVICE ID",
                                                        },
                                    reembed_table = {"TEST - CLIENT":True,
                                                    "TEST - CLIENT REQUEST":True,
                                                    "TEST - FLIGHTS":True,
                                                    "TEST - ACCOMODATIONS":True,
                                                    "TEST - ACTIVITIES":True,
                                                    "TEST - SERVICES":True,
                                                    }
                                                    
                                    )


loading specialist: TRAVELLER ...
TRAVELLER embedding: TEST - CLIENT - LOADED
TRAVELLER embedding: TEST - CLIENT REQUEST - LOADED
TRAVELLER embedding: TEST - FLIGHTS - LOADED
TRAVELLER embedding: TEST - ACCOMODATIONS - LOADED
TRAVELLER embedding: TEST - ACTIVITIES - LOADED
TRAVELLER embedding: TEST - SERVICES - LOADED
TRAVELLER loaded


## _*Generate*_ : travel package from customer/agent prompt  + inventory

In [7]:
# Ask Traveller to generate a travel package
initial_query = "Hafeez from Kuala Lumpur wants to go Bali for 4 days for 2 pax. Budget: $2000. I want to chill at the beach"

convo_package = librarian.Traveller.III_generate_travel_package(initial_query = initial_query,
                                                                 topN = 6, 
                                                                 model_name = "gemini-pro",
                                                                 )

Token size of the prompt for cl100k_base ~ 3745
**Summary**

Immerse yourself in the beauty of Bali on this 4-day beach getaway designed to soothe your soul and rejuvenate your senses. Dive into the pristine waters, bask in the warm sun, indulge in luxurious spa treatments, and explore the island's stunning natural wonders. This trip_recommendation offers a perfect blend of relaxation, adventure, and cultural experiences, ensuring an unforgettable escape to paradise.

**Journey Highlights**

* Chill out on the white-sand beaches of Bali
* Pamper yourself with a rejuvenating spa treatment
* Embark on a scenic hiking and cliff-hunting tour
* Savor delicious meals at local restaurants
* Capture the beauty of Bali's rice fields
* Your journey takes you to: Kuala Lumpur - Denpasar Bali - Kuala Lumpur

**Itinerary**

**Day 1**

* 0900hrs: Depart from Kuala Lumpur International Airport (FLIGHT ID: 1)
* 1100hrs: Arrive at Denpasar Bali Airport (DPS)
* 1200hrs: Transfer to Hard Rock Hotel Bali 

## Regenerate : travel package from followup prompt + inventory

In [10]:
followup_query = "Give me the cheapest bali activities instead of that 4.5K hunting tour"

convo_package = librarian.Traveller.III_generate_travel_package(initial_query = "",
                                                                followup_query = followup_query,
                                                                 topN = 6, 
                                                                 model_name = "gemini-pro",
                                                                 )

Token size of the prompt for cl100k_base ~ 3857
**Refined Travel Package**

**Summary**

Immerse yourself in the beauty of Bali on this tailored 4-day beach getaway designed to soothe your soul and rejuvenate your senses. Dive into the pristine waters, bask in the warm sun, indulge in luxurious spa treatments, and explore the island's stunning natural wonders. This trip offers a perfect blend of relaxation, adventure, and cultural experiences, ensuring an unforgettable escape to paradise.

**Journey Highlights**

* Chill out on the white-sand beaches of Bali
* Pamper yourself with a rejuvenating spa treatment
* Embark on a scenic hiking and cliff-hunting tour
* Savor delicious meals at local restaurants
* Capture the beauty of Bali's rice fields
* Your journey takes you to: Kuala Lumpur - Denpasar Bali - Kuala Lumpur

**Itinerary**

**Day 1**

* 0900hrs: Depart from Kuala Lumpur International Airport (FLIGHT ID: 1)
* 1100hrs: Arrive at Denpasar Bali Airport (DPS)
* 1200hrs: Transfer to

## OCR : extract text from image

### a) EasyOCR(image: jpg)

In [4]:
import easyocr

def extract_text_from_image(image_file):
  """
  Extracts text and calculates average probability from an image using EasyOCR.

  Args:
      image_file: Path to the image file.

  Returns:
      A tuple containing:
          * Extracted text as a single string concatenated from all blocks.
          * Raw output from EasyOCR (list of tuples with text, probability, and bounding box).
          * Average probability of all text blocks.
  """

  # Create a reader for the languages you want to support
  reader = easyocr.Reader(['en'])

  # Use the reader to read the text from the image
  raw = reader.readtext(image_file)

  # Extract text and probabilities
  text = ' '.join([block[1] for block in raw])
  probabilities = [block[2] for block in raw]

  # Calculate average probability (handling potential division by zero)
  average_probability = sum(probabilities) / len(probabilities) if probabilities else 0.0

  return text, raw, average_probability

# Example usage
# image_file = './database/travel/Ingest/Cust_req_1.jpg'
image_path = './database/travel/Ingest/TP_Cxxx_20240105_KL1.jpg'
text, raw, avg_prob = extract_text_from_image(image_path)
print("Extracted Text:", text)
# print("Raw Data:", raw)
print("Average Probability:", avg_prob)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Extracted Text: exil Ieo @ su-nig {6v.m4 One- no } | 03 2114 5414 Ajeat 54nia Trave( "V^+ I(tuc SMART Travel Agency :Sedunia Travel Services WeRLD Package Kuala Lumpur Tour Date 05 January 2024 Kuala Lumpur Tour No. of Pax 04 pax (02 adult & 02 child) Date 05 January 2024 11.OOAM 7 OOPM Full Day (8 Hours) Time Pick up Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kualo Lumpur, Drop off Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kuala Lumpur, Type of Mini Van Vehicle Tour Guide English Speaking Entrance Fee Not included Meals Adult (MYR 50.00) Child (MYR 25.00) etarian Meal for Indian family (MYR) MYR 550.00 (transport/tour) MYR 300 (Tour guide) MYR 300 (meals) MYR3OO Price (markup) Highlight Gaze at the beautiful city of Kuala Lumpur from the Observation Deck of the Petronas Twin Towers Climb Batu Caves to explore Southeast Asia'$ most visited Hindu Temple Capture your

In [5]:
text

'exil Ieo @ su-nig {6v.m4 One- no } | 03 2114 5414 Ajeat 54nia Trave( "V^+ I(tuc SMART Travel Agency :Sedunia Travel Services WeRLD Package Kuala Lumpur Tour Date 05 January 2024 Kuala Lumpur Tour No. of Pax 04 pax (02 adult & 02 child) Date 05 January 2024 11.OOAM 7 OOPM Full Day (8 Hours) Time Pick up Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kualo Lumpur, Drop off Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kuala Lumpur, Type of Mini Van Vehicle Tour Guide English Speaking Entrance Fee Not included Meals Adult (MYR 50.00) Child (MYR 25.00) etarian Meal for Indian family (MYR) MYR 550.00 (transport/tour) MYR 300 (Tour guide) MYR 300 (meals) MYR3OO Price (markup) Highlight Gaze at the beautiful city of Kuala Lumpur from the Observation Deck of the Petronas Twin Towers Climb Batu Caves to explore Southeast Asia\'$ most visited Hindu Temple Capture your memories with

### b) LLM(image: jpg)

In [7]:
image_path = './database/travel/Ingest/TP_Cxxx_20240105_KL1.jpg'
prompt_1 = """The image is a travel itinerary that i want to digitise so that i can populate a database. 
Help analyse the content and reproduce it in a structured format such that it is easily extractable
"""
response = librarian.Traveller.model_specialist.prompt_image(image_path = image_path,
                                                  prompt_1 = prompt_1,
                                                  prompt_2 = None,
                                                  model_name = "gemini-pro-vision",)

In [11]:
print(response.text)

 ```python
{
  "travel_agency": "Sedunia Travel Services",
  "package": "1 Day Kuala Lumpur Tour",
  "date": "05 January 2024",
  "no_of_pax": 4,
  "children": 2,
  "time": "11:00AM - 7:00PM",
  "duration": "8 Hours",
  "pick_up_location": "Alia Hotel in Bangsar",
  "drop_off_location": "58, Jalan Ang Seng, Brickfields, 50470 Kuala Lumpur, Wilayah Persekutuan Kuala Lumpur",
  "type_of_vehicle": "Mini Van",
  "guide": "English Speaking",
  "meals": {
    "adult": 50,
    "child": 25,
    "vegetarian_meal_for_indian_family": 30
  },
  "entrance_fee": {
    "adult": 55,
    "child": 30
  },
  "transport_fee": 30,
  "tour_guide_fee": 300,
  "total_price": 1450,
  "highlights": [
    "Gaze at the beautiful city of Kuala Lumpur from the Observation Deck of the KLCC Twin Towers",
    "Capture your memories with your camera at all thes most visited Hindu Temple in Central Market, one of the oldest markets in Kuala Lumpur",
    "Shop in Central Market",
    "Petronas Twin Towers",
    "Menara K

### d) TELEGRAM BOT
Run app_digitiser.py to start the telegram bot 

This requires 

python
pip install python-telegram-bot